In [94]:
import sqlite3
import pandas as pd

In [95]:
conn = sqlite3.connect('collection.anki21')
cursor = conn.cursor()

## Get the different prompts

In [156]:
# spanish <-> catalan verbs
cat_sp_df = pd.read_csv('spanish_to_catalan_verbs.tsv', sep='\t',
                        encoding='latin1')
cat_sp_df = cat_sp_df[['spanish_verb', 'catalan_verb']]
cat_sp_df.rename({'catalan_verb': 'inf_verb'}, axis=1, inplace=True)

In [157]:
# get the tags and notes for each card
cursor.execute("SELECT sfld, tags FROM notes")
rows = cursor.fetchall()
notes = []
tags = []
intro_phrases = ["El verbo en",
                 "Before",
                 "During",
                 "Grammar",
                 "For ", 
                 "I hesitated",
                 "If you see"]
for row in rows:
    if row[0].startswith(intro_phrases[0]): pass
    elif row[0].startswith(intro_phrases[1]): pass
    elif row[0].startswith(intro_phrases[2]): pass
    elif row[0].startswith(intro_phrases[3]): pass
    elif row[0].startswith(intro_phrases[4]): pass
    elif row[0].startswith(intro_phrases[5]): pass
    elif row[0].startswith(intro_phrases[6]): pass
    else:
        notes.append(row[0])
        tags.append(row[1])

In [158]:
# make into a df
df = pd.DataFrame(data=notes,
                  columns=['note'])
df['tags'] = tags

# get the base verb and add catalan equivalent
# (inner merge to get rid of verbs I didn't add equivalents for)
df['spanish_verb'] = df['note'].str.split('::…', expand=True)[1].str.split('…}}', expand=True)[0]
df = df.merge(cat_sp_df,
              how='inner',
              on='spanish_verb')

In [159]:
df.tags[1]

' ends_in_er extreme_irregularity irregular_verb participio regular_form ser '

In [160]:
# get rid of tú_vos
print(len(df.index))
df = df.loc[~df.tags.str.contains(' tú_vos ')]
print(len(df.index))

3761
2991


In [161]:
# get rid of subjunctive future
print(len(df.index))
df = df.loc[~df.tags.str.contains(' subjuntivo_futuro ')]
print(len(df.index))

2991
2666


In [102]:
# tags I care about that I can merge on 

# 1, 2, 3rd person
person_map = {'yo': '1',
              'tú': '2',
              'él_ella_usted': '3',
              'nosotros': '1',
              'vosotros': '2',
              'ellos_ellas_ustedes': '3'}
df['person'] = '0'
for key, item in person_map.items():
    thing = ' '+key+' '
    df.loc[df.tags.str.contains(thing), 'person'] = person_map[key]
    
# singular or plural
sing_or_plural_map = {'yo': 'S',
              'tú': 'S',
              'él_ella_usted': 'S',
              'nosotros': 'P',
              'vosotros': 'P',
              'ellos_ellas_ustedes': 'P'}
df['sing_or_plural'] = '0'
for key, item in sing_or_plural_map.items():
    thing = ' '+key+' '
    df.loc[df.tags.str.contains(thing), 'sing_or_plural'] = sing_or_plural_map[key]
df.loc[df.tags.str.contains(' participio '), 'sing_or_plural'] = 'S'

# mood
mood_map = {'presente': 'I',
           'imperfecto': 'I',
           'indefinido': 'I',
           'futuro': 'I',
           'condicional': 'I',
           'subjuntivo_presente': 'S',
           'subjuntivo_pasado': 'S',
           'imperativo': 'M',
           'negative_imperativo': 'M',
           'gerundio': 'G',
           'participio': 'P'}
df['mood'] = '0'
for key, item in mood_map.items():
    thing = ' '+key+' '
    df.loc[df.tags.str.contains(thing), 'mood'] = mood_map[key]

# tense
tense_map = {'presente': 'P',
           'imperfecto': 'I',
           'indefinido': 'passat_perifrastic',
           'futuro': 'F',
           'condicional': 'C',
           'subjuntivo_presente': 'P',
           'subjuntivo_pasado': 'I'}
df['tense'] = '0'
for key, item in tense_map.items():
    thing = ' '+key+' '
    df.loc[df.tags.str.contains(thing), 'tense'] = tense_map[key]

# pos_neg_cmd
pos_neg_map = {'imperativo': 'pos',
               'negative_imperativo': 'neg'}
df['pos_neg_cmd'] = '0'
for key, item in pos_neg_map.items():
    thing = ' '+key+' '
    df.loc[df.tags.str.contains(thing), 'pos_neg_cmd'] = pos_neg_map[key]

In [103]:
# TODO add a version of the past using "ahir" 
# for the passat simple (ie he anat)

In [ ]:
# split the note up so I can translate and replace shit
# by the different tense / mood combos
split_dict = {'gerundio': ['Mientras estoy estudiando…', 1],
              'participio': ['Hugo ha', 1],
              'presente': ['⊙ Ahora mismo, ⊙', 1],
              'imperfecto': ['⇠ En esa época, a menudo, ⇠', 1],
              'indefinido': ['↧ En aquel momento ↧', 1],
              'futuro': ['→ En el futuro, →', 1],
              'condicional': ['…', 2],
              'subjuntivo_presente': ['〰 Resulta divertido que 〰', 1],
              'subjuntivo_pasado': ['↫ Fue sorprendente que ↫', 1],
              'imperativo': ['¡', 1]}
# tag:negative_imperativo}


               
               

In [152]:
df.loc[df.note.str.contains('esto')].tail()
# df.loc[(df.mood=='M')&(df.pos_neg_cmd=='neg')].head()
# df.loc[df.note.str.startswith('Mujer, por favor, no')].head()
df.loc[(df.pos_neg_cmd=='neg')].head()

,note,tags,spanish_verb,inf_verb,person,sing_or_plural,mood,tense,pos_neg_cmd,note1,note2,note1_1,note1_2


In [124]:
spanish_phrases = []
df['note1'] = df.note.str.split('{{', expand=True)[0]
df['note1_1'] = df.note1.str.split('…', expand=True)[0]
df['note1_2'] = df.note1.str.split('…', expand=True)[1]
df['note2'] = df.note.str.split('}}', expand=True)[1]

In [129]:
print(len(df.index))
print(len(df.note1_1.unique()))
# spanish_phrases += 
df.note1_1.unique().tolist()

2666
55


['Mientras estoy estudiando',
 '',
 '→ En el futuro, →nosotros ',
 '→ En el futuro, →yo ',
 '→ En el futuro, →vosotros ',
 '→ En el futuro, →ellos/ellas/ustedes ',
 '→ En el futuro, →él/ella/usted ',
 '⇠ En esa época, a menudo, ⇠nosotros ',
 '⇠ En esa época, a menudo, ⇠yo ',
 '⇠ En esa época, a menudo, ⇠vosotros ',
 '⇠ En esa época, a menudo, ⇠ellos/ellas/ustedes ',
 '⇠ En esa época, a menudo, ⇠él/ella/usted ',
 '⊙ Ahora mismo, ⊙nosotros ',
 '⊙ Ahora mismo, ⊙yo ',
 '⊙ Ahora mismo, ⊙vosotros ',
 '⊙ Ahora mismo, ⊙ellos/ellas/ustedes ',
 '⊙ Ahora mismo, ⊙él/ella/usted ',
 '↧ En aquel momento ↧nosotros ',
 '↧ En aquel momento ↧yo ',
 '↧ En aquel momento ↧vosotros ',
 '↧ En aquel momento ↧ellos/ellas/ustedes ',
 '↧ En aquel momento ↧él/ella/usted ',
 'Vámonos equipo, por favor,¡ (que) ',
 'Tú y tú, por favor,¡ ',
 'Señores, por favor,¡ (que) ',
 'Señora, por favor,¡ (que) ',
 'Hugo ha ',
 '↫ Fue sorprendente que ↫nosotros ',
 '↫ Fue sorprendente que ↫yo ',
 '↫ Fue sorprendente que ↫vosotros

In [130]:
temp = df.loc[df.mood=='G']
print(len(df.index))
print(len(df.note1_2.unique()))
# spanish_phrases += 
df.note1_2.unique().tolist()

2666
18


['ella está ',
 ' Si pasara algo, ',
 ' Si ocurriese esto, ',
 ' Si aconteciese esto, ',
 ' Si se produjera esto, ',
 ' Si acaeciera algo, ',
 None,
 'ella se está ',
 'ella me está ',
 'no está ',
 'el público está ',
 'la pitón está ',
 'ella no está ',
 'está ',
 'el programa está ',
 'la pera está ',
 'la comunidad se está ',
 'él se está ']

In [131]:
temp = df.loc[df.mood=='G']
print(len(df.index))
print(len(df.note2.unique()))
df.note2.unique()
# spanish_phrases += 
df.note2.unique()

2666
386


array([' (en el teatro)', ' (en consecuencia), ya !(nosotros)',
       ' (en consecuencia), ya !(vosotros)',
       ' (en consecuencia), ya ! (ellos/ellas/ustedes)',
       ' (en consecuencia), ya !(él/ella/usted)', ' (a la situación)',
       ' (la velocidad al tráfico)',
       ' (la velocidad al tráfico), ya !(nosotros)',
       ' (la velocidad al tráfico), ya !(vosotros)',
       ' (la velocidad al tráfico), ya ! (ellos/ellas/ustedes)',
       ' (la velocidad al tráfico), ya !(él/ella/usted)', ' (libros)',
       ' (más libros), ya !(nosotros)', ' (más libros), ya !(vosotros)',
       ' (más libros), ya ! (ellos/ellas/ustedes)',
       ' (más libros), ya !(él/ella/usted)', ' (cariño)', ' (el tren)',
       ' (el tren), ya !(nosotros)', ' (el tren), ya !(vosotros)',
       ' (el tren), ya ! (ellos/ellas/ustedes)',
       ' (el tren), ya !(él/ella/usted)', ' (un edificio)',
       ' (la cabeza)', ' (la cabeza), ya !(nosotros)',
       ' (la cabeza), ya !(vosotros)',
       ' (la cabe

In [ ]:
# translated spanish to cat. phrases to stick on the cards

phrase_dict = {'Mientras estoy estudiando': 'A mesura que estic estudiant'

In [111]:
# take a look at shit
df = df.sort_values(by=merge_cols)
df.to_csv('temp.tsv', sep='\t')

In [109]:
# temp = df.loc[df.mood=='G']
# print(len(temp.index))
# print(len(temp.note1.unique()))

65
12


In [110]:
# temp.note1.unique()

array(['Mientras estoy estudiando…ella está ',
       'Mientras estoy estudiando…está ',
       'Mientras estoy estudiando…ella me está ',
       'Mientras estoy estudiando…ella se está ',
       'Mientras estoy estudiando…ella no está ',
       'Mientras estoy estudiando…el programa está ',
       'Mientras estoy estudiando…él se está ',
       'Mientras estoy estudiando…no está ',
       'Mientras estoy estudiando…la pera está ',
       'Mientras estoy estudiando…la comunidad se está ',
       'Mientras estoy estudiando…la pitón está ',
       'Mientras estoy estudiando…el público está '], dtype=object)

In [ ]:
## testing merge w/ cat conj. w/ fer

In [87]:
# add the catalan conjugations
cat_conj_df = pd.read_csv('fer_parsed.tsv', sep='\t')


In [88]:
# just hacer<->fer for now
df = df.loc[df.inf_verb=='fer']

In [89]:
len(df.index)

41

In [90]:
cat_conj_df.head()
merge_cols = ['inf_verb','mood', 'tense', 'person',
              'sing_or_plural', 'pos_neg_cmd']
for c in merge_cols:
    df[c] = df[c].astype(str)
    cat_conj_df[c] = cat_conj_df[c].astype(str)
df = df.merge(cat_conj_df, 
              how='left', 
              on=merge_cols)
print(len(df.index))

52


In [91]:
df.loc[df.conj_verb.isnull()].head(2)[merge_cols]

,inf_verb,mood,tense,person,sing_or_plural,pos_neg_cmd


,note,tags,spanish_verb,inf_verb,person,sing_or_plural,mood,tense,pos_neg_cmd,conj_verb,code,region,gender,haver_verb,anar_verb
0,Mientras estoy estudiando…ella está {{c1::haci...,ends_in_er extreme_irregularity gerundio hace...,hacer,fer,0,0,G,0,0,fent,VMG00000,central,0,NaN,NaN
1,Hugo ha {{c1::hecho::…hacer…}} (sus ejercicios...,ends_in_er extreme_irregularity hacer in_the_...,hacer,fer,0,S,P,0,0,fet,VMP00SM0,central,M,NaN,NaN
2,Hugo ha {{c1::hecho::…hacer…}} (sus ejercicios...,ends_in_er extreme_irregularity hacer in_the_...,hacer,fer,0,S,P,0,0,feta,VMP00SF0,central,F,NaN,NaN
3,"⊙ Ahora mismo, ⊙yo {{c1::hago::…hacer…}} (past...",ends_in_er extreme_irregularity hacer irregul...,hacer,fer,1,S,I,P,0,faig,VMIP1S00,central,0,NaN,NaN
4,"⊙ Ahora mismo, ⊙él/ella/usted {{c1::hace::…hac...",ends_in_er extreme_irregularity hacer in_the_...,hacer,fer,3,S,I,P,0,fa,VMIP3S00,central,0,NaN,NaN
